# buy and hold 전략
1. 구매 후 일정 기간이 지나면 판매

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('../csv/AAPL.csv', index_col='Date')
df

In [ ]:
# 결측치를 확인 
df.info()

In [ ]:
# Volume의 데이터가 결측치인 데이터를 출력
df.loc[~df["Volume"].isna()]

In [ ]:
df.drop(["1981-08-10", "1981-08-21"], axis=0)

In [ ]:
# 결측치와 양의 무한대, 음의 무한대의 값을 제외한 데이터만 출력
df.loc[df['Open'] != -np.inf]
df.loc[df['Open'] != np.inf]
df.loc[~df['Open'].isna()]

In [ ]:
df.loc[
    (df['Open'] != -np.inf) & (df['Open'] != np.inf) & (~df['Open'].isna())
]

In [ ]:
flag = ~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)

In [ ]:
df = df.loc[flag]

In [ ]:
# df의 종가를 제외한 데이터를 모두 삭제
df[['Close']]

In [ ]:
price_df = df.loc[:, ['Close']]

In [ ]:
# 일별 수익율 = (오늘의 종가 - 전날의 종가) / 전날의 종가
# 전 행의 데이터를 로드하려면? shift(n)
# case1
(price_df['Close'] - price_df['Close'].shift(1)) / price_df['Close'].shift(1)


In [ ]:
# case2
price_df['Close'].diff() / price_df['Close'].shift()


In [ ]:
# case3
price_df['daily_rtn'] = price_df['Close'].pct_change()

In [ ]:
price_df

In [ ]:
# 일별 수익율을 가지고 총 수익율 계산
test_df = price_df.iloc[300:600]
test_df

In [ ]:
rtn = 1
# rtn * (1 + daily_rtn)

for i in test_df['daily_rtn'].tolist():
    rtn = rtn * (1 + i)

print(rtn)


In [ ]:
(1 + test_df['daily_rtn']).cumprod()

In [ ]:
# (매도한 날의 종가 - 매수한 날의 종가) / 매수한 날의 종가
(test_df.iloc[-1]['Close'] - test_df.iloc[0]['Close']) / test_df.iloc[0]['Close']

In [ ]:
## buyandhold 함수로 생성
## 매개변수를 3개
## 데이터프레임 , 구매한 날, 판매한 날
def bnh(_df, _start, _end):
    # index의 데이터를 시계열로 변경
    _df.index = pd.to_datetime(_df.index, format='%Y-%m-%d')
    # 결측치와 무한대의 값을 제외시키는 부분
    _df = _df.loc[ ~_df.isin([np.nan, np.inf, -np.inf]).any(axis=1) ]
    # 종가를 제외한 나머지 컬럼을 모두 삭제
    _df = _df[['Close']]
    # 일별 수익율 컬럼를 생성
    _df['daily_rtn'] = _df['Close'].pct_change()
    # 구매한 날짜와 판매한 날짜를 기준으로 데이터를 필퍼( 행의 조건으로 필터 )
    _df = _df.loc[_start : _end]
    # 누적 수익율 컬럼을 생성
    _df['st_rtn'] = (1 + _df['daily_rtn']).cumprod()

    return _df

In [ ]:
# 문자열 데이터를 시계열 변경
from datetime import datetime

In [ ]:
df = pd.read_csv("../csv/AMZN.csv", index_col='Date')
df

In [ ]:
start = '20000102'
end = '20151231'
# 문자열을 시계열 변경
start = datetime.strptime(start, '%Y%m%d').isoformat()
end = datetime.strptime(end, '%Y%m%d').isoformat()

In [ ]:
bnh(df, start, end)